<a href="https://colab.research.google.com/github/hellomstars/Kaggle_Titanic/blob/main/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import random as rnd

In [ ]:
train_df=pd.read_csv("./titanic/train.csv")
test_df=pd.read_csv("./titanic/test.csv")

In [ ]:
print(train_df.columns.values)
print(test_df.columns.values)

['PassengerId' 'Survived' 'Pclass' 'Name' 'Sex' 'Age' 'SibSp' 'Parch'
 'Ticket' 'Fare' 'Cabin' 'Embarked']
['PassengerId' 'Pclass' 'Name' 'Sex' 'Age' 'SibSp' 'Parch' 'Ticket' 'Fare'
 'Cabin' 'Embarked']


In [ ]:
train_df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [ ]:
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:
#변수 제거
train_data = train_df.drop(['Ticket', 'Cabin'], axis=1)
test_data = test_df.drop(['Ticket', 'Cabin'], axis=1)
combine = [train_data, test_data]

train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S


In [ ]:
#이름 추출
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

pd.crosstab(train_data['Title'], train_data['Sex'])

Sex,female,male
Title,,
Capt,0,1
Col,0,2
Countess,1,0
Don,0,1
Dr,1,6
Jonkheer,0,1
Lady,1,0
Major,0,2
Master,0,40


In [ ]:
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    
train_data[['Title', 'Age']].groupby(['Title'], as_index=False).mean()

,Title,Age
0,Capt,70.000000
1,Col,58.000000
2,Countess,33.000000
3,Don,40.000000
4,Dr,42.000000
5,Jonkheer,38.000000
6,Lady,48.000000
7,Major,48.500000
8,Master,4.574167
9,Miss,21.845638


In [ ]:
Title_age_mean = train_data[['Title', 'Age']].groupby(['Title'], as_index=False).mean()

In [ ]:
int(Title_age_mean.loc[Title_age_mean.Title == 'Mr', 'Age'])

32

In [ ]:
#비어있는 나이 채워넣기
for dataset in combine:
    dataset.loc[ (dataset.Age.isnull()) & (dataset.Title == 'Mr'),'Age'] = int(Title_age_mean.loc[Title_age_mean.Title == 'Mr', 'Age'])
    dataset.loc[ (dataset.Age.isnull()) & (dataset.Title == 'Mrs'),'Age'] = int(Title_age_mean.loc[Title_age_mean.Title == 'Mrs', 'Age'])
    dataset.loc[ (dataset.Age.isnull()) & (dataset.Title == 'Miss'),'Age'] = int(Title_age_mean.loc[Title_age_mean.Title == 'Miss', 'Age'])
    dataset.loc[ (dataset.Age.isnull()) & (dataset.Title == 'Master'),'Age'] = int(Title_age_mean.loc[Title_age_mean.Title == 'Master', 'Age'])
    dataset.loc[ (dataset.Age.isnull()) & (dataset.Title == 'Dr'),'Age'] = int(Title_age_mean.loc[Title_age_mean.Title == 'Dr', 'Age'])

In [ ]:
train_data.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       2
Title          0
dtype: int64

In [ ]:
#선착장 유실 정보 채워넣기
for dataset in combine:
    dataset.loc[ (dataset.Embarked.isnull()),'Embarked'] = 'S'
    
train_data.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
Title          0
dtype: int64

In [ ]:
#가족 수 따라서
for dataset in combine:
    dataset['FamilyNum'] = dataset['SibSp'] + dataset['Parch'] + 1

train_data[['FamilyNum', 'Survived']].groupby(['FamilyNum'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,FamilyNum,Survived
3,4,0.724138
2,3,0.578431
1,2,0.552795
6,7,0.333333
0,1,0.303538
4,5,0.200000
5,6,0.136364
7,8,0.000000
8,11,0.000000


In [ ]:
#혼자인지 가족이 있는지 여부 확인
for dataset in combine:
    dataset['isAlone'] = 0
    dataset.loc[dataset['FamilyNum'] == 1, 'isAlone'] = 1

train_data[['isAlone', 'Survived']].groupby('isAlone', as_index = False).mean()

,isAlone,Survived
0,0,0.505650
1,1,0.303538


In [ ]:
train_set = train_data.drop(['Name', 'Parch', 'SibSp', 'FamilyNum'], axis=1)
test_set = test_data.drop(['Name', 'Parch', 'SibSp', 'FamilyNum'], axis=1)
combine = [train_set, test_set]
train_set.head()

,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked,Title,isAlone
0,1,0,3,male,22.0,7.2500,S,Mr,0
1,2,1,1,female,38.0,71.2833,C,Mrs,0
2,3,1,3,female,26.0,7.9250,S,Miss,1
3,4,1,1,female,35.0,53.1000,S,Mrs,0
4,5,0,3,male,35.0,8.0500,S,Mr,1


In [ ]:
#Fare 정규화

def normal(data):
    
    data = (data - data.min())/(data.max() - data.min())
    
    return data

Fare_scaled = normal(train_set['Fare'])
Fare_scaled

0      0.014151
1      0.139136
2      0.015469
3      0.103644
4      0.015713
         ...   
886    0.025374
887    0.058556
888    0.045771
889    0.058556
890    0.015127
Name: Fare, Length: 891, dtype: float64

In [ ]:
for dataset in combine:
    dataset['Fare'] = Fare_scaled

In [ ]:
train_set.head()

,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked,Title,isAlone
0,1,0,3,male,22.0,0.014151,S,Mr,0
1,2,1,1,female,38.0,0.139136,C,Mrs,0
2,3,1,3,female,26.0,0.015469,S,Miss,1
3,4,1,1,female,35.0,0.103644,S,Mrs,0
4,5,0,3,male,35.0,0.015713,S,Mr,1


In [ ]:
train_set.to_csv("Titanic train 전처리_안서현.csv", index = True)
test_set.to_csv("Titanic test 전처리_안서현.csv", index = True)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier